In [ ]:
import sys

sys.path.append("..")

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

from openpoints.utils import EasyConfig, set_random_seed, dist_utils
from openpoints.transforms import build_transforms_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.loss import build_criterion_from_cfg
from openpoints.models import build_model_from_cfg

import torch
from torch_geometric.data import Data, Batch
import os
import glob

In [ ]:
from torch_scatter import scatter
from openpoints.utils import set_random_seed, save_checkpoint, load_checkpoint, resume_checkpoint, setup_logger_dist, \
    cal_model_parm_nums, Wandb, generate_exp_directory, resume_exp_directory, EasyConfig, dist_utils, find_free_port, load_checkpoint_inv
from openpoints.utils import AverageMeter, ConfusionMatrix, get_mious
from openpoints.dataset import build_dataloader_from_cfg, get_features_by_keys, get_class_weights
from openpoints.dataset.data_util import voxelize
from openpoints.dataset.semantic_kitti.semantickitti import load_label_kitti, load_pc_kitti, remap_lut_read, remap_lut_write, get_semantickitti_file_list
from openpoints.transforms import build_transforms_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.loss import build_criterion_from_cfg
from openpoints.models import build_model_from_cfg

In [ ]:
cfg = EasyConfig()
cfg.load("../cfgs/s3dis/pointnextpyg-xl.yaml", recursive=True)
cfg.dataset.common.data_root = os.path.join("../", cfg.dataset.common.data_root)

In [ ]:
model = build_model_from_cfg(cfg.model)
model = model.to(torch.device("cuda:0"))

In [ ]:
num_points = 16000

pos = torch.randn(num_points, 3)

x = None

data = Data(pos=pos, x=pos)

batch = Batch.from_data_list([data, data])
batch = batch.to(torch.device("cuda:0"))

In [ ]:
cfg.distributed = False

In [ ]:
cfg.dataset.return_geometric_data = False

In [ ]:
val_loader = build_dataloader_from_cfg(cfg.get('val_batch_size', cfg.batch_size),
                                           cfg.dataset,
                                           cfg.dataloader,
                                           datatransforms_cfg=cfg.datatransforms,
                                           split='val',
                                           distributed=cfg.distributed
                                           )

In [ ]:
data

In [ ]:
from openpoints.dataset.build import build_dataset_from_cfg

dataset = build_dataset_from_cfg(cfg.dataset.common)
dataset[2]

In [ ]:
data = next(iter(val_loader))
data

In [ ]:
cfg.feature_keys

In [ ]:
data.heights.shape

In [ ]:
data.x.shape

In [ ]:
model(data.to('cuda')).shape

In [ ]:
from openpoints.dataset.data_util import get_features_by_keys

x = get_features_by_keys(data, cfg.feature_keys)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(tmp["pos"][0][:, 0], tmp["pos"][0][:, 1], tmp["pos"][0][:, 2], s=1)
plt.show()

In [ ]:
tmp["x"]

In [ ]:
model(batch).shape